In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
#pd.set_option('max_colwidth', 0)
import warnings
warnings.filterwarnings("ignore")

from cplex import *
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.optimizers import COBYLA,SLSQP,L_BFGS_B
from qiskit import QuantumRegister
from qiskit.utils import algorithm_globals
from qiskit_aer.primitives import Sampler


input_file_name = "Optimising Fleet Allocation - Input Data - v1.0.xlsx"
df_PlannedScheduleO = pd.read_excel(input_file_name, 'PlannedSchedule')
print(df_PlannedScheduleO.head(10))
print(df_PlannedScheduleO.shape[0])

#Data Correction
df_PlannedScheduleO.loc[df_PlannedScheduleO["Flight Number"]== 1115 , "Arrival Airport"] = "MAN"
df_PlannedScheduleO.loc[df_PlannedScheduleO["Flight Number"]== 1167 , "Arrival Airport"] = "MAN"
df_PlannedScheduleO.loc[df_PlannedScheduleO["Flight Number"]== 1190 , "Arrival Airport"] = "MAN"
df_PlannedScheduleO.loc[df_PlannedScheduleO["Flight Number"]== 1193 , "Arrival Airport"] = "MAN"
print(df_PlannedScheduleO.head(10))

df_PlannedScheduleO["Departure Date"]  = pd.to_datetime(df_PlannedScheduleO["Departure Date"])
df_PlannedScheduleO['day of week'] = df_PlannedScheduleO['Departure Date'].dt.dayofweek
df_PlannedScheduleO["Departure Date"]  = df_PlannedScheduleO["Departure Date"].dt.strftime('%d-%m-%Y')
df_PlannedScheduleO['day of week'] = df_PlannedScheduleO['day of week'] + 1


print(df_PlannedScheduleO.head())

list_Dates = list(df_PlannedScheduleO["Departure Date"].unique())
print(list_Dates)
print(len(list_Dates))


df_Aircraft = pd.read_excel(input_file_name, 'Aircraft')
print(df_Aircraft.head())
print(df_Aircraft.shape[0])
df_Aircraft["DOW"] = df_Aircraft["DOW"].astype("string")


dict_DOWAircrafts = {}

for d in range(1,8):
    listAircraftsReg = []
    for i in range(0, df_Aircraft.shape[0]):
        if df_Aircraft["DOW"][i][d-1] == str(d):
            listAircraftsReg.append(df_Aircraft["AircraftReg"][i])
            
    dict_DOWAircrafts[d] = listAircraftsReg
    
    
print(dict_DOWAircrafts)

df_PlannedMaintenance = pd.read_excel(input_file_name, 'PlannedMaintainance')
print(df_PlannedMaintenance.head(10))
print(df_PlannedMaintenance.shape[0])


df_PlannedMaintenance.loc[df_PlannedMaintenance["AircraftReg"]== "G-BF", "EndTime"] = 1600
print(df_PlannedMaintenance.head(10))


df_PlannedMaintenance["FromDate"]  = pd.to_datetime(df_PlannedMaintenance["FromDate"]).dt.strftime('%d-%m-%Y')
df_PlannedMaintenance["ToDate"]  = pd.to_datetime(df_PlannedMaintenance["ToDate"]).dt.strftime('%d-%m-%Y')
print(df_PlannedMaintenance.head(10))


df_PlannedMaintenance["ListDatesMaintenance"] = ""
for i in range(0, df_PlannedMaintenance.shape[0]):
    df_PlannedMaintenance["ListDatesMaintenance"][i] = [ d.strftime('%d-%m-%Y') for d in pd.date_range(df_PlannedMaintenance["FromDate"][i],df_PlannedMaintenance["ToDate"][i]) ]
    #pd.date_range(df_PlannedMaintenance["FromDate"][i],df_PlannedMaintenance["ToDate"][i]).astype(str).tolist()
    
print(df_PlannedMaintenance.head())


dict_DateAircraftsMaint = {}

for d in list_Dates:
    listAircraftsMaint = []
    for i in range(0, df_PlannedMaintenance.shape[0]):
        if d in df_PlannedMaintenance["ListDatesMaintenance"][i]:
            listAircraftsMaint.append(df_PlannedMaintenance["AircraftReg"][i])
            
    dict_DateAircraftsMaint[d] = listAircraftsMaint    
    
print(dict_DateAircraftsMaint)


def funcInttoMins(a) :
    time = str(a)
    if len(time) == 4:
        h  = time[len(time)-4:len(time)-2]
        
    if len(time) == 3:
        h  = time[len(time)-3:len(time)-2]
        
    m = time[len(time)-2:len(time)]
    Mins = int(h)*60 + int(m)
    
    return Mins

def funcMinstoInt(a) :
    time = divmod(a,60)
    #print(time)
    if len(str(time[1])) ==1 :
        Int = int(str(time[0])+ '0'+ str(time[1]))
        
    if len(str(time[1])) == 2 :
        Int = int(str(time[0]) + str(time[1]))
    
    return Int

#################################################################################


AircraftCostQuantum = 0
AbsDifferenceinSeatsQuantum = 0
SurplusQuantum = 0
DeficitQuantum = 0
NumofAllocatedFlightsQ = 0
NumofEmptyFlightsQ = 0
MaxFlightsQ = 0

AircraftCostClassical = 0
AbsDifferenceinSeatsClassical = 0
SurplusClassical = 0
DeficitClassical = 0
NumofAllocatedFlights = 0
NumofEmptyFlights = 0
MaxFlights = 0

df_OutputQuantumFull = pd.DataFrame(columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time', 'Departure Date'])
df_OutputClassicalFull = pd.DataFrame(columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time', 'Departure Date'])

#list_Dates = list_Dates[0:2]  #  45278 '29-11-2023'
#print("list_Dates=", list_Dates)
#Loop starts
#d_index = 10

for d_index in range(0, len(list_Dates)):

    df_PlannedSchedule = df_PlannedScheduleO[df_PlannedScheduleO["Departure Date"]== list_Dates[d_index]] 
    print(df_PlannedSchedule.head())
    df_PlannedSchedule.reset_index(inplace = True, drop = True)
    print(df_PlannedSchedule.head())
    print(df_PlannedSchedule.shape[0])


    list(df_PlannedSchedule["day of week"].unique())[0]


    #listAircrafts = list(df_Aircraft["AircraftReg"])
    listAircraftsN = dict_DOWAircrafts[list(df_PlannedSchedule["day of week"].unique())[0]]
    print(listAircraftsN)
    print("Num of Aircrafts=",len(listAircraftsN))


    dict_FlightNoOrigin =dict(zip(df_PlannedSchedule["Flight Number"], df_PlannedSchedule["Departure Airport"]))
    dict_FlightNoDestination =dict(zip(df_PlannedSchedule["Flight Number"], df_PlannedSchedule["Arrival Airport"]))
    dict_FlightNoDepartureTime =dict(zip(df_PlannedSchedule["Flight Number"], df_PlannedSchedule["Depature Time"]))
    dict_FlightNoArrivalTime =dict(zip(df_PlannedSchedule["Flight Number"], df_PlannedSchedule["Arrival Time"]))

    dict_AircraftCost =dict(zip(df_Aircraft.AircraftReg, df_Aircraft.OperatingCost))
    dict_AircraftCapacity =dict(zip(df_Aircraft.AircraftReg, df_Aircraft.NumSeats))
    dict_AircraftMaxFlights= dict(zip(df_Aircraft.AircraftReg, df_Aircraft["Maximum allowable flights"]))
    dict_AircraftBase= dict(zip(df_Aircraft.AircraftReg, df_Aircraft["Nightly Base"]))



    list_AircraftBases = list(df_Aircraft["Nightly Base"].unique())
    print("list_AircraftBases =", list_AircraftBases)


    k = 0 
    r = 0
    df_Routes = pd.DataFrame(columns = ['RouteNumber', 'RouteName','BaseAirport', 'RouteStartTime', 'RouteEndTime', 'NumofFlights', 'RouteForecastedSeats'])

    for i in range(df_PlannedSchedule.shape[0]):
        #print("check i=", i)
        if df_PlannedSchedule["Leg Number"][i] == 1:
            First = df_PlannedSchedule["Departure Airport"][i] 
            RouteStartTime = df_PlannedSchedule["Depature Time"][i] 
            RouteName = ''
            #RouteName = str(df_PlannedSchedule["Flight Number"][i])
            listFS = []
            listRouteFlights = []
            #listFS.append(df_PlannedSchedule["Forecasted Seats"][i])
        if i != (df_PlannedSchedule.shape[0]-1):
            listFS.append(df_PlannedSchedule["Forecasted Seats"][i])
            #Route_FS = max(listFS)
            #Route_FS = math.ceil(np.mean(listFS))
            Route_FS = np.sum(listFS)
            listRouteFlights.append(df_PlannedSchedule["Flight Number"][i])
            RouteName = RouteName + str(df_PlannedSchedule["Flight Number"][i]) + "_"
            if df_PlannedSchedule["Leg Number"][i+1] == 1 : 
                Last = df_PlannedSchedule["Arrival Airport"][i]
                RouteEndTime = df_PlannedSchedule["Arrival Time"][i] 
                r+=1
                    
                df_extended = pd.DataFrame([[r , RouteName, First, RouteStartTime, RouteEndTime, len(listFS),  Route_FS, listRouteFlights], ], columns = ['RouteNumber', 'RouteName','BaseAirport', 'RouteStartTime', 'RouteEndTime', 'NumofFlights', 'RouteForecastedSeats', 'RoutesFlightsList'])
                df_Routes = pd.concat([df_Routes, df_extended], ignore_index=True)
                
                if First != Last :
                    print("i =", i)
                    print("First = ", First)
                    print("Last = ", Last)
                    k+=1
                    
        if i == (df_PlannedSchedule.shape[0]-1) :
            #print("checkloop")
            listFS.append(df_PlannedSchedule["Forecasted Seats"][i])
            #Route_FS = max(listFS)
            #Route_FS = math.ceil(np.mean(listFS))
            Route_FS = np.sum(listFS)
            listRouteFlights.append(df_PlannedSchedule["Flight Number"][i])
            RouteName = RouteName + str(df_PlannedSchedule["Flight Number"][i])
            r+=1

            Last = df_PlannedSchedule["Arrival Airport"][i]
            RouteEndTime = df_PlannedSchedule["Arrival Time"][i]
                    
            df_extended = pd.DataFrame([[r , RouteName, First, RouteStartTime, RouteEndTime,  len(listFS),  Route_FS, listRouteFlights], ], columns = ['RouteNumber', 'RouteName','BaseAirport', 'RouteStartTime', 'RouteEndTime', 'NumofFlights', 'RouteForecastedSeats', 'RoutesFlightsList'])
            df_Routes = pd.concat([df_Routes, df_extended], ignore_index=True)
            
            #print("Same Aircraft base mismatches =", k)
            print(df_Routes)
            #print(df_Routes.shape[0])
            #print(df_Routes["BaseAirport"].unique())



    df_Routes.head(20)

    df_Routes['RouteStartTime'] = df_Routes['RouteStartTime'].map(funcInttoMins)
    df_Routes['RouteEndTime'] = df_Routes['RouteEndTime'].map(funcInttoMins)

    print(df_Routes)

    listRoutes = list(df_Routes["RouteNumber"]) 
    print(listRoutes)
    print("Num of Routes=",len(listRoutes))

    dict_RouteNumFlights= dict(zip(df_Routes.RouteNumber, df_Routes.NumofFlights))
    dict_RouteDemand=dict(zip(df_Routes.RouteNumber, df_Routes.RouteForecastedSeats))
    dict_RouteStartTime=dict(zip(df_Routes.RouteNumber, df_Routes.RouteStartTime))
    dict_RouteEndTime=dict(zip(df_Routes.RouteNumber, df_Routes.RouteEndTime))

    dict_BaseAirport=dict(zip(df_Routes.RouteNumber, df_Routes.BaseAirport))
    dict_RouteFlightsList=dict(zip(df_Routes.RouteNumber, df_Routes.RoutesFlightsList))


    listAircraftsM = dict_DateAircraftsMaint[list_Dates[d_index]]
    print(listAircraftsM)

    #Aircraft  maintenance constraint 
    listAircrafts = [item for item in listAircraftsN if item not in listAircraftsM]
    print(listAircrafts)
    print("Num of Aircrafts=",len(listAircrafts))


    #Aircraft DOW constraint
    df_AircraftN = df_Aircraft[df_Aircraft['AircraftReg'].isin(listAircrafts)]
    print(df_AircraftN.head())
    print(df_AircraftN.shape[0])

    dictAircraftRoute = {}
    for a in listAircrafts:
        for r in listRoutes:
            dictAircraftRoute[a,r] = 0    

    List_MaxAllowableFlights = list(df_AircraftN["Maximum allowable flights"].unique())
    print(List_MaxAllowableFlights)

    List_NumofFlights = list(df_Routes["NumofFlights"].unique())
    print(List_NumofFlights)
    #bHeuristics = 0
    bMILP = 1

    Beta = 100
    Gamma = 100000

    df_Output = pd.DataFrame(columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time'])

    #ListEmptyFlights = []

    if bMILP == 1:
        #Building Binary Model 
        mdl = Model('RouteAssignmentModel')

        #populate decision variables
        vars = {}
        t1 = mdl.linear_expr()


    for r in List_NumofFlights:    
        print("r =", r)
        df_RoutesR = df_Routes[df_Routes["NumofFlights"]== r]
        if r == 6 : #Preprocessing - for route =6, there should be minimum routes available for aircrafts (Observed from data)
            if len(list(df_RoutesR[df_RoutesR["BaseAirport"]== "LTN"]["RouteNumber"])) <3 :
                df_RoutesC  = df_RoutesR[df_RoutesR["BaseAirport"]== "LGW"]
                ListRoutesC = list(df_RoutesC["RouteNumber"])
                ListRoutesCh = ListRoutesC[0:4]
                #print("Check =", ListRoutesCh)

                df_RoutesR.reset_index(drop=True, inplace=True)

                for rc in ListRoutesCh : 
                    for i in range(0, df_RoutesR.shape[0]):
                        if df_RoutesR["RouteNumber"][i] == rc :
                            df_RoutesR["BaseAirport"][i] = "LTN"

                            dict_FlightNoOrigin["Emp" + str(df_RoutesR["RoutesFlightsList"][i][0])] = dict_FlightNoOrigin[df_RoutesR["RoutesFlightsList"][i][0]]
                            dict_FlightNoOrigin["Emp" + str(df_RoutesR["RoutesFlightsList"][i][-1])] = dict_FlightNoOrigin[df_RoutesR["RoutesFlightsList"][i][-1]]

                            dict_FlightNoDestination["Emp" + str(df_RoutesR["RoutesFlightsList"][i][0])] = dict_FlightNoDestination[df_RoutesR["RoutesFlightsList"][i][0]]
                            dict_FlightNoDestination["Emp" + str(df_RoutesR["RoutesFlightsList"][i][-1])] = dict_FlightNoDestination[df_RoutesR["RoutesFlightsList"][i][-1]]

                            dict_FlightNoDepartureTime["Emp" + str(df_RoutesR["RoutesFlightsList"][i][0])] = dict_FlightNoDepartureTime[df_RoutesR["RoutesFlightsList"][i][0]]
                            dict_FlightNoDepartureTime["Emp" + str(df_RoutesR["RoutesFlightsList"][i][-1])] = dict_FlightNoDepartureTime[df_RoutesR["RoutesFlightsList"][i][-1]]

                            dict_FlightNoArrivalTime["Emp" + str(df_RoutesR["RoutesFlightsList"][i][0])] = dict_FlightNoArrivalTime[df_RoutesR["RoutesFlightsList"][i][0]]
                            dict_FlightNoArrivalTime["Emp" + str(df_RoutesR["RoutesFlightsList"][i][-1])] = dict_FlightNoArrivalTime[df_RoutesR["RoutesFlightsList"][i][-1]]

                            #ListEmptyFlights.append("Emp" + str(df_RoutesR["RoutesFlightsList"][i][0]))
                            #ListEmptyFlights.append("Emp" + str(df_RoutesR["RoutesFlightsList"][i][-1]))

                            df_RoutesR["RoutesFlightsList"][i][0] = "Emp" + str(df_RoutesR["RoutesFlightsList"][i][0])
                            df_RoutesR["RoutesFlightsList"][i][-1] = "Emp" + str(df_RoutesR["RoutesFlightsList"][i][-1])
                    
                
        #df_RoutesR = df_Routes[df_Routes["NumofFlights"]== r]
        print(df_RoutesR.head(20))
        List_RoutesR =list(df_RoutesR["RouteNumber"])
        print(List_RoutesR)

        if r == 6:
            df_AircraftsR = df_AircraftN[(df_AircraftN["Maximum allowable flights"] == 6) | (df_AircraftN["Maximum allowable flights"] == 7)]
        elif r == 4:
            df_AircraftsR = df_AircraftN[(df_AircraftN["Maximum allowable flights"] == 5) | (df_AircraftN["Maximum allowable flights"] == 4)]
        elif r == 3:
            df_AircraftsR = df_AircraftN[(df_AircraftN["Maximum allowable flights"] == 3)]

        print(df_AircraftsR.head(20))
        List_AircraftsR =list(df_AircraftsR["AircraftReg"])
        print(List_AircraftsR)
        
        for b in list_AircraftBases:
            
            df_AircraftsB = df_AircraftsR[df_AircraftsR["Nightly Base"]== b]
            df_RoutesB  = df_RoutesR[df_RoutesR["BaseAirport"]== b]
            
    #         ###########################
    #         if bHeuristics == 1 : #Heuristically solve problem

    #             df_AircraftsC = df_AircraftsB.sort_values('NumSeats')
    #             df_RoutesC = df_RoutesB.sort_values('RouteForecastedSeats')

    #             #print(df_AircraftsC.head())
    #             #print(df_RoutesC.head())

    #             ListAircraftsC =list(df_AircraftsC["AircraftReg"])
    #             ListRoutesC =list(df_RoutesC["RouteNumber"])

    #             print(ListAircraftsC)
    #             print(ListRoutesC)

    #             if len(ListAircraftsC)>0:
    #                 for i in range(0, len(ListAircraftsC)):
    #                     dictAircraftRoute[ListAircraftsC[i],ListRoutesC[i]] = 1

    #             for a in List_AircraftsR:
    #                 list_AircraftFlights = []
    #                 for r in List_RoutesR:
    #                     if dictAircraftRoute[a,r]== 1:
    #                         print("a =", a + " r =", r)
    #                         for f in dict_RouteFlightsList[r]:
    #                             list_AircraftFlights.append(f)

    #                 for f in list_AircraftFlights:                  
    #                     AircraftTail= a
    #                     FlightNo = f
    #                     Origin = dict_FlightNoOrigin[f]
    #                     Dest = dict_FlightNoDestination[f]
    #                     DepartureTime = dict_FlightNoDepartureTime[f]
    #                     ArrivalTime = dict_FlightNoArrivalTime[f]
    #                     df_extended = pd.DataFrame([[AircraftTail , 0, FlightNo, Origin, Dest, DepartureTime, ArrivalTime], ], columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time'])
    #                     df_Output = pd.concat([df_Output, df_extended], ignore_index=True)
                                        
    #         ##################################################                     
            if bMILP == 1:   
                
                ListAircraftsB =list(df_AircraftsB["AircraftReg"])
                ListRoutesB =list(df_RoutesB["RouteNumber"])
                
                for r in ListRoutesB:
                    for a in ListAircraftsB:
                        vars[(a, r)] = mdl.binary_var(name=f'x_{a}_{r}')
                
                #print("vars = ", vars)
                
                for r in ListRoutesB:
                    for a in ListAircraftsB:
                        #t1.add_term(vars[(a, r)], (dict_RouteNumFlights[r] *dict_AircraftCost[a] + Beta*(dict_RouteDemand[r]-dict_RouteNumFlights[r] *dict_AircraftCapacity[a])))
                        t1.add_term(vars[(a,r)], (dict_RouteNumFlights[r] *dict_AircraftCost[a] + Beta*(dict_RouteDemand[r]-dict_RouteNumFlights[r] *dict_AircraftCapacity[a]) - Gamma ))
                        #t1.add_term(vars[(a,r)], (Beta*(dict_RouteDemand[r]-dict_RouteNumFlights[r] *dict_AircraftCapacity[a]) - Gamma))
                        #t1.add_term(vars[(a,r)], (- Gamma))   #check for Idle time

                #constraint - each route flown by atmost one aircraft 
                for r in ListRoutesB:
                    c1 = mdl.linear_expr()
                    for a in ListAircraftsB:
                        c1.add_term(vars[(a, r)], 1)
                    mdl.add_constraint(c1 <= 1, "Rtcovering-" + str(r))

                #constraint - each aircraft is assigned exacly one route 
                for a in ListAircraftsB:
                    c2 = mdl.linear_expr()
                    for r in ListRoutesB:
                        c2.add_term(vars[(a, r)], 1)
                    mdl.add_constraint(c2 == 1, "Accovering-" + str(a))
                    
                    #####################################

    if bMILP == 1:                
        mdl.minimize(t1)
        #Print model details and solve using CPLEX 
        print(mdl.export_as_lp_string())
        mdl.print_information()     
        
        #Solving Binary Model 
        mdl.solve()

        mdl.print_solution()
        
        for a in listAircrafts:
            for r in listRoutes:
                if (a,r) in vars.keys():
                    if vars[(a,r)].solution_value  == 1 :
                        dictAircraftRoute[a,r] = 1
                    
        for a in listAircrafts:
            list_AircraftFlights = []
            for r in listRoutes:
                if dictAircraftRoute[a,r]== 1:
                    #print("a =", a + " r =", r)
                    for f in dict_RouteFlightsList[r]:
                        list_AircraftFlights.append(f)

            for f in list_AircraftFlights:                  
                AircraftTail= a
                FlightNo = f
                Origin = dict_FlightNoOrigin[f]
                Dest = dict_FlightNoDestination[f]
                DepartureTime = dict_FlightNoDepartureTime[f]
                ArrivalTime = dict_FlightNoArrivalTime[f]
                df_extended = pd.DataFrame([[AircraftTail , 0, FlightNo, Origin, Dest, DepartureTime, ArrivalTime], ], columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time'])
                df_Output = pd.concat([df_Output, df_extended], ignore_index=True)
                
                #################################

    print(df_Output.head(10))

    df_Output = df_Output.sort_values(['AircraftTail', 'Initial Departure Time'])
    df_Output = df_Output.reset_index(drop=True)
    print(df_Output.head(10))


    j = 0
    for i in range(0, df_Output.shape[0]-1):
        if df_Output["AircraftTail"][i] == df_Output["AircraftTail"][i+1] :
            j+= 1
            df_Output["Sequence"][i] = j
        else:
            j+= 1
            df_Output["Sequence"][i] = j
            j = 0

    if df_Output["AircraftTail"][df_Output.shape[0]-1] == df_Output["AircraftTail"][df_Output.shape[0]-2] :
        df_Output["Sequence"][df_Output.shape[0]-1] = df_Output["Sequence"][df_Output.shape[0]-2]+1
    else:
        df_Output["Sequence"][df_Output.shape[0]-1] = 1

    #len(list(df_Output['FlightNo'].unique()))
        
    df_Output["Departure Date"] = ""
    df_Output["Departure Date"] = list_Dates[d_index]
    print(df_Output.head(20))

    for a in listAircrafts:
        for r in listRoutes:
            if dictAircraftRoute[a,r] == 1:
                AircraftCostClassical += dict_RouteNumFlights[r] *dict_AircraftCost[a] 
                MaxFlights += dict_AircraftMaxFlights[a]
                AbsDifferenceinSeatsClassical += abs(dict_RouteNumFlights[r] *dict_AircraftCapacity[a] -dict_RouteDemand[r])

                if dict_RouteNumFlights[r] *dict_AircraftCapacity[a] > dict_RouteDemand[r] :
                    SurplusClassical += (dict_RouteNumFlights[r] *dict_AircraftCapacity[a] - dict_RouteDemand[r])

                if dict_RouteDemand[r] > dict_RouteNumFlights[r] *dict_AircraftCapacity[a]  :
                    DeficitClassical += (dict_RouteDemand[r] - dict_RouteNumFlights[r] *dict_AircraftCapacity[a])

    DemandFlightsList = list(df_PlannedSchedule["Flight Number"])
    AllocatedwEmptFlightsList = list(df_Output['FlightNo'])
    AllocatedFlightsList = list(set(DemandFlightsList).intersection(AllocatedwEmptFlightsList))
    NumofAllocatedFlights += len(AllocatedFlightsList)
    NumofEmptyFlights += len(AllocatedwEmptFlightsList) -len(AllocatedFlightsList)

    df_OutputClassicalFull = pd.concat([df_OutputClassicalFull, df_Output], ignore_index=True, sort=False)

    ###################################################################################################
    #Quantum Starts

    beta = Parameter("β")

    #Building QUBO model
    vars['ancilla_{0}'] = mdl.binary_var(name='ancilla_{0}')
    b_removeconstraint  = 1

    if b_removeconstraint == 1 :
        for r in listRoutes:
            mdl.remove_constraint("Rtcovering-" + str(r))

        for a in listAircrafts:
            mdl.remove_constraint("Accovering-" + str(a))

    print(mdl.export_as_lp_string())
    mdl.print_information()
    #translate a docplex.mp model into a quadratic program
    mdl_q = from_docplex_mp(mdl)


    #convert quadractic constraint optimization to QUBO
    conv = QuadraticProgramToQubo()
    mdl_qubo = conv.convert(mdl_q)

    print("-----------QUBO FORMULATION-------------")
    print(mdl_qubo.export_as_lp_string())

    num_bv_qubo = mdl_qubo.get_num_binary_vars()
    bv_qubo = mdl_qubo.variables_index     # key = variable name, value = variable index

    print(num_bv_qubo)
    print(bv_qubo)

    betaList = []
    for i in range(0, mdl_qubo.get_num_binary_vars()-1):
        betaList.append(Parameter("β" + str(i)))
        
    print(betaList)


    #populate quantum resisters
    qrs = {}
    for var in bv_qubo:
        qrs[var] = QuantumRegister(1, f'q_{bv_qubo[var]}')
        
    ######build initial feasible state 
    init_qc= QuantumCircuit(name= "ini_qc")
    for var in list(qrs.keys()):
        init_qc.add_register(qrs[var])

    print(qrs)
    #init_qc.draw(output='text') 

    #####build rotation mixer  circuit 
    rotation_mixer = QuantumCircuit(name= "rotation_mixer")
    for var in list(qrs.keys()):
        rotation_mixer.add_register(qrs[var])

    b=0
    for var in list(qrs.keys()):
        if var != 'ancilla_{0}':
            rotation_mixer.rx(betaList[b], qrs[var])
            b+=1
            #rotation_mixer.rx(beta, qrs[var])
        
    #rotation_mixer.draw(output='text') 

    # define a few contrl_bit_flip_circuit - cbf 
    cbf3_qc = QuantumCircuit(3, name ='cbf3')  #0 = ii, 1 = ji, 2 = jj, 3 = ancilla
    cbf3_qc.ccx(0,1,2)
    cbf3_qc.cx(2,1)
    #cbf3_qc.cx(3,2)
    cbf3_qc_gate = cbf3_qc.to_gate()
    cbf3_qc.draw(output='text')


    #### controlled bit flip mixer - 1 -
    cntrl_bit_flip_mixer_qc = QuantumCircuit(name= "cntrl_bit_flip_mixer_qc")
    for var in list(qrs.keys()):
        cntrl_bit_flip_mixer_qc.add_register(qrs[var])
        
    for a in listAircrafts:
        var_subset = []
        for r in listRoutes:
            #var = (a,r)
            var = f'x_{a}_{r}'
            if var in qrs:
                var_subset.append(var)

        var_pairs =[]
        
        for k in range(0, len(var_subset)):
            for l in range(0, len(var_subset)):
                if l> k:
                    var_pairs.append((var_subset[k], var_subset[l]))

        #print("var_pairs aircrafts =", var_pairs)

        for p in range(len(var_pairs)):

            cntrl_bit_flip_mixer_qc.append(cbf3_qc_gate, [qrs[var_pairs[p][0]], qrs[var_pairs[p][1]], qrs['ancilla_{0}']])
            #cntrl_bit_flip_mixer_qc.x(qrs[var_pairs[p][0])
            cntrl_bit_flip_mixer_qc.reset(qrs['ancilla_{0}'])


    for r in listRoutes:
        var_subset = []
        for a in listAircrafts :
            #var = (a,r)
            var = f'x_{a}_{r}'
            if var in qrs:
                var_subset.append(var)

    
        var_pairs =[]
        
        for k in range(0, len(var_subset)):
            for l in range(0, len(var_subset)):
                if l> k:
                    var_pairs.append((var_subset[k], var_subset[l]))

        #print("var_pairs routes =", var_pairs)

        for p in range(len(var_pairs)):

            cntrl_bit_flip_mixer_qc.append(cbf3_qc_gate, [qrs[var_pairs[p][0]], qrs[var_pairs[p][1]], qrs['ancilla_{0}']])
            #cntrl_bit_flip_mixer_qc.x(qrs[var_pairs[p][0])
            cntrl_bit_flip_mixer_qc.reset(qrs['ancilla_{0}'])


    cntrl_bit_flip_mixer_qc.draw(output='text')

    combined_mixer_qc = rotation_mixer
    #combined_mixer_qc = cntrl_bit_flip_mixer_qc
    combined_mixer_qc.append(cntrl_bit_flip_mixer_qc, cntrl_bit_flip_mixer_qc.qubits)
    combined_mixer_qc.draw(output='text') 

    ###########################
    method = 'matrix_product_state'

    num_shots = 20
    num_reps = 1
    seed = 100

    algorithm_globals.random_seed = seed
    optimizers = [COBYLA(disp = True), SLSQP(maxiter=100, disp = True), L_BFGS_B(maxiter=100)] #maxiter= 30 #reps = 0.1
    opt_indx = 0  #0 = COBYLA, 1 = SLSQP, 2 = L_BFGS_B, derived from above populated list
    sampler = Sampler(backend_options={'method':method,'shots':num_shots}, run_options={'shots':num_shots, 'seed':seed})
    #########################

    print("Solving QAOA ...")

    qaoa_mes = QAOA(
        sampler, 
        optimizer=optimizers[opt_indx], 
        initial_state=init_qc,
        reps=num_reps,
        mixer=combined_mixer_qc,
        #initial_point = [0,0]
        )

    qaoa = MinimumEigenOptimizer(qaoa_mes)
    result_qaoa = qaoa.solve(mdl_qubo)

    listlastparam = result_qaoa.min_eigen_solver_result.optimal_point
    listlastoptimal_parameters = result_qaoa.min_eigen_solver_result.optimal_parameters
    print(listlastoptimal_parameters)
    print()
    #print(listlastparam[0])
    #exit ()

    for sample in result_qaoa.samples[:5]:
        print(sample)

    def format_qaoa_samples(samples, max_len: int = 10):
        qaoa_res = []
        for s in samples:
            qaoa_res.append(("".join([str(int(_)) for _ in s.x]), s.fval, s.probability))
        res = sorted(qaoa_res, key=lambda x: x[-2])[0:max_len]
        return [(_[0] + f": value: {_[1]:.3f}, probability: {1e2*_[2]:.1f}%") for _ in res]

    result_samples = format_qaoa_samples(result_qaoa.samples)
    for sample in result_samples:
        print(sample)

    print(result_qaoa.prettyprint())



    print("QOAO Result")
    print(result_qaoa.status)
    print('Obj val=', result_qaoa.fval)
    for k,v  in result_qaoa.variables_dict.items():
        if v != 0 :
            print(f'{k}={v}') 


    num_InfeasibleConst= 0
    #constraint - each flight flown by exacly (atmost) one aircraft 
    for r in listRoutes:
        num_allocF = 0
        for a in listAircrafts:
            if (a, r) in vars and result_qaoa.variables_dict[f'x_{a}_{r}'] != 0:
                num_allocF +=1
                
        if num_allocF > 1:
            print("ERR: Rtcovering constraint for Route" + str(r))
            num_InfeasibleConst +=1
            
    #constraint - only one aircraft is assigned to a route on overlap routes
    for a in listAircrafts:
        num_allocA = 0
        for r in listRoutes:
            if (a, r) in vars and result_qaoa.variables_dict[f'x_{a}_{r}'] != 0:
                num_allocA +=1
                
                
        if num_allocA > 1:
            print("ERR: AcCovering constraint for Aircraft" + str(a))
            num_InfeasibleConst +=1
            
    if num_InfeasibleConst == 0:
        print("All constraints are feasible")


    df_OutputQ = pd.DataFrame(columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time'])

    dictAircraftRouteQ = {}
    for a in listAircrafts:
        for r in listRoutes:
            dictAircraftRouteQ[(a,r)] = 0

    for k,v  in result_qaoa.variables_dict.items():
        if v != 0 :
            var = (str(k.split('_')[1]), int(k.split('_')[2]))
            #print("var =", var)
            dictAircraftRouteQ[(var)] = 1


    #print(dictAircraftRouteQ)

    for a in listAircrafts:
        list_AircraftFlights = []
        for r in listRoutes:
            if dictAircraftRouteQ[(a,r)]== 1:
                for f in dict_RouteFlightsList[r]:
                    list_AircraftFlights.append(f)

        for f in list_AircraftFlights:                  
            AircraftTail= a
            FlightNo = f
            Origin = dict_FlightNoOrigin[f]
            Dest = dict_FlightNoDestination[f]
            DepartureTime = dict_FlightNoDepartureTime[f]
            ArrivalTime = dict_FlightNoArrivalTime[f]
            df_extended = pd.DataFrame([[AircraftTail , 0, FlightNo, Origin, Dest, DepartureTime, ArrivalTime], ], columns = ['AircraftTail', 'Sequence', 'FlightNo', 'Origin', 'Destination', 'Initial Departure Time', 'Initial Arrival Time'])
            df_OutputQ = pd.concat([df_OutputQ, df_extended], ignore_index=True)
                

    df_OutputQ = df_OutputQ.sort_values(['AircraftTail', 'Initial Departure Time'])
    df_OutputQ = df_OutputQ.reset_index(drop=True)
    print(df_OutputQ.head(10))

    j = 0
    for i in range(0, df_OutputQ.shape[0]-1):
        if df_OutputQ["AircraftTail"][i] == df_OutputQ["AircraftTail"][i+1] :
            j+= 1
            df_OutputQ["Sequence"][i] = j
        else:
            j+= 1
            df_OutputQ["Sequence"][i] = j
            j = 0

    if df_OutputQ["AircraftTail"][df_OutputQ.shape[0]-1] == df_OutputQ["AircraftTail"][df_OutputQ.shape[0]-2] :
        df_OutputQ["Sequence"][df_OutputQ.shape[0]-1] = df_OutputQ["Sequence"][df_OutputQ.shape[0]-2]+1
    else:
        df_OutputQ["Sequence"][df_OutputQ.shape[0]-1] = 1


    #len(list(df_OutputQ['FlightNo'].unique()))
    #len(list(df_OutputQ['AircraftTail'].unique()))
        
    df_OutputQ["Departure Date"] = ""
    df_OutputQ["Departure Date"] = list_Dates[d_index]
    print(df_OutputQ.head(20))

    for a in listAircrafts:
        for r in listRoutes:
            if dictAircraftRouteQ[a,r] == 1:
                AircraftCostQuantum += dict_RouteNumFlights[r] *dict_AircraftCost[a] 
                MaxFlightsQ += dict_AircraftMaxFlights[a]
                AbsDifferenceinSeatsQuantum += abs(dict_RouteNumFlights[r] *dict_AircraftCapacity[a] -dict_RouteDemand[r])

                if dict_RouteNumFlights[r] *dict_AircraftCapacity[a] > dict_RouteDemand[r] :
                    SurplusQuantum += (dict_RouteNumFlights[r] *dict_AircraftCapacity[a] - dict_RouteDemand[r])

                if dict_RouteDemand[r] > dict_RouteNumFlights[r] *dict_AircraftCapacity[a]  :
                    DeficitQuantum += (dict_RouteDemand[r] - dict_RouteNumFlights[r] *dict_AircraftCapacity[a])
    
    DemandFlightsListQ = list(df_PlannedSchedule["Flight Number"])
    AllocatedwEmptFlightsListQ = list(df_OutputQ['FlightNo'])
    AllocatedFlightsListQ = list(set(DemandFlightsListQ).intersection(AllocatedwEmptFlightsListQ))
    NumofAllocatedFlightsQ += len(AllocatedFlightsListQ)
    NumofEmptyFlightsQ += len(AllocatedwEmptFlightsListQ) -len(AllocatedFlightsListQ)


    df_OutputQuantumFull = pd.concat([df_OutputQuantumFull, df_OutputQ], ignore_index=True, sort=False)

################################################################################
print("Quantum Results :")
df_OutputQuantumFull.to_csv("OutputQuantum.csv")

print("NumofAllocatedFlightsQ =", NumofAllocatedFlightsQ)
print("NumofEmptyFlightsQ =", NumofEmptyFlightsQ)
print("AircraftCostQuantum =", AircraftCostQuantum)
print("AbsDifferenceinSeatsQuantum =", AbsDifferenceinSeatsQuantum)
print("SurplusQuantum =", SurplusQuantum)
print("DeficitQuantum =", DeficitQuantum)
print("MaxFlightsQ =", MaxFlightsQ)

print("\n")
print("Classical Results :")

df_OutputClassicalFull.to_csv("OutputClassical.csv")

print("NumofAllocatedFlights =", NumofAllocatedFlights)
print("NumofEmptyFlights =", NumofEmptyFlights)
print("AircraftCostClassical =", AircraftCostClassical)
print("AbsDifferenceinSeatsClassical =", AbsDifferenceinSeatsClassical)
print("SurplusClassical =", SurplusClassical)
print("DeficitClassical =", DeficitClassical)
print("MaxFlights =", MaxFlights)
